In [ ]:
import hopsworks
project = hopsworks.login()
fs = project.get_feature_store()

In [ ]:
fv = fs.get_feature_view("fraud_model", 1)

In [ ]:
X_train, X_test, y_train, y_test = fv.get_train_test_split(1)

In [ ]:
X_train

In [ ]:
import numpy as np
import xgboost as xgb

clf = xgb.XGBClassifier()

clf.fit(X_train, y_train)

In [ ]:
y_pred_test = clf.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score
# Compute f1 score
metrics = {"f1score": f1_score(y_test, y_pred_test, average='micro')}
metrics

In [ ]:
# Hopsworks Schema registry
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

In [ ]:
import os
import joblib 

model_dir="fraud_online_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

joblib.dump(clf, model_dir + '/xgboost_fraud_online_model.pkl')

In [ ]:
mr = project.get_model_registry()

fraud_model = mr.python.create_model(
    name="xgboost_fraud_online_model", 
    metrics=metrics,
    model_schema=model_schema,
    description="Fraud Online Predictor")

fraud_model.save(model_dir)